In [1]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [2]:
# %% 1. การติดตั้งและ Import Library

# ติดตั้ง Library ที่จำเป็น
!pip install -q transformers torch accelerate sentence-transformers lightgbm scikit-learn

import pandas as pd
import numpy as np
import json
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tqdm.auto import tqdm
import torch
# ตั้งค่าอุปกรณ์
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"อุปกรณ์ที่ใช้: {DEVICE}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.9 MB/s eta 0:00:00
อุปกรณ์ที่ใช้: cuda


In [3]:
# %% 2. การเตรียมข้อมูล

print("กำลังโหลด STS-B Dataset เพื่อจำลองข้อมูล...")
dataset = load_dataset("PhilipMay/stsb_multi_mt", "en")

# --- สร้างข้อมูลสำหรับฝึกซ้อม (train.csv) ---
train_df = pd.DataFrame({
    'resume_text': dataset['train']['sentence1'],
    'job_description_text': dataset['train']['sentence2'],
    'match_score': dataset['train']['similarity_score']
})

# --- สร้างข้อมูลสำหรับทดสอบ (test.csv) ---
test_df = pd.DataFrame({
    'resume_text': dataset['test']['sentence1'],
    'job_description_text': dataset['test']['sentence2'],
    # เก็บค่าจริงไว้เพื่อประเมินผล
    'actual_score': dataset['test']['similarity_score']
})
test_df['pair_id'] = test_df.index

# ลดขนาดข้อมูลเพื่อความรวดเร็วในการทดลอง (ในการแข่งจริงให้ใช้ทั้งหมด)
train_df = train_df.sample(n=500, random_state=42)
test_df = test_df.sample(n=200, random_state=42)


print(f"เตรียมข้อมูล train_df สำเร็จ จำนวน {len(train_df)} ตัวอย่าง")
print(train_df.head())

print(f"\nเตรียมข้อมูล test_df สำเร็จ จำนวน {len(test_df)} ตัวอย่าง")
print(test_df.head())


กำลังโหลด STS-B Dataset เพื่อจำลองข้อมูล...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/470k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/108k [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/142k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1500 [00:00<?, ? examples/s]

เตรียมข้อมูล train_df สำเร็จ จำนวน 500 ตัวอย่าง
                                            resume_text  \
4034                More than 30 striking miners killed   
5241     Sony Xperia Z2 and Nokia‚Äôs X series unveiled   
2201          He's not wealthy because he's successful.   
4211       Three more US soldiers killed in Afghanistan   
2671  The songs are on offer for 99 cents each, or $...   

                                   job_description_text  match_score  
4034  More than 30 striking miners killed in South A...        3.800  
5241  Court orders political ban on Italy‚Äôs Berlus...        0.000  
2201       Why hate people because they are successful?        0.800  
4211                 NATO Soldier Killed in Afghanistan        1.800  
2671  The company will offer songs for 99 cents and ...        3.333  

เตรียมข้อมูล test_df สำเร็จ จำนวน 200 ตัวอย่าง
                                           resume_text  \
599  A group of people are sitting at at a beach on...   
881  Go

In [4]:
train_df

,resume_text,job_description_text,match_score
4034,More than 30 striking miners killed,More than 30 striking miners killed in South A...,3.800
5241,Sony Xperia Z2 and Nokia‚Äôs X series unveiled,Court orders political ban on Italy‚Äôs Berlus...,0.000
2201,He's not wealthy because he's successful.,Why hate people because they are successful?,0.800
4211,Three more US soldiers killed in Afghanistan,NATO Soldier Killed in Afghanistan,1.800
2671,"The songs are on offer for 99 cents each, or $...",The company will offer songs for 99 cents and ...,3.333
...,...,...,...
4420,China stocks close higher after economic meeting,China stocks close higher on Wednesday,3.400
1057,A woman holding a small baby.,A smiling woman holding a small baby.,4.000
2653,But U.S. troops will not shrink from mounting ...,But American troops will not shrink from mount...,4.750
4406,6.9 magnitude earthquake hits New Zealand,5.6 magnitude earthquake shakes Iran,0.200


In [5]:
test_df

,resume_text,job_description_text,actual_score,pair_id
599,A group of people are sitting at at a beach on...,A group of people are sitting at a beach watch...,4.20,599
881,Governor Gray Davis estimated yesterday that t...,State officials estimated the cost at nearly $...,3.50,881
634,You need to read a lot to know what you like a...,You have to decide how much you want to demand...,0.00,634
425,Three children wrestle on a floral carpet.,Three children playing on the floor.,3.80,425
906,Q: Can I buy coverage for prescription drugs r...,Congress has added a new benefit - an option t...,1.25,906
...,...,...,...,...
1356,Pak religious body endorses underage marriage,CII endorses underage marriage,3.00,1356
844,There are two things to consider:,I think there are two important things to cons...,5.00,844
56,A man puts three pieces of meat into a pan.,A man is putting meat in a pan.,4.00,56
931,"""These despicable acts were committed by kille...",These despicable acts were committed by killer...,3.50,931


In [6]:
# %% 3. การโหลดโมเดล LLM และ Tokenizer

model_id = "Qwen/Qwen2-7B-Instruct"
print(f"กำลังโหลด Tokenizer และโมเดล LLM: '{model_id}'...")

# โหลด Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
# โหลด Model
model_llm = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16, # ใช้ bfloat16 เพื่อลดการใช้ VRAM
    device_map="auto"
)

print("โหลดโมเดล LLM สำเร็จ")


กำลังโหลด Tokenizer และโมเดล LLM: 'Qwen/Qwen2-7B-Instruct'...


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

โหลดโมเดล LLM สำเร็จ


In [8]:
# %% 4. การออกแบบ Prompt แบบ Chain-of-Thought (CoT)

def create_cot_prompt(resume, job_description):
    """
    สร้าง Prompt ที่สมบูรณ์ตามรูปแบบ Chain-of-Thought
    """
    prompt_template = f"""คุณคือ HR ที่เชี่ยวชาญ จงประเมินว่าเรซูเม่นี้เหมาะกับตำแหน่งงานนี้แค่ไหน

ตำแหน่งงาน: "{job_description}"
เรซูเม่: "{resume}"

คิดวิเคราะห์เป็นขั้นตอนดังนี้:
1. ทักษะหลักที่ตำแหน่งงานต้องการคืออะไร?
2. ผู้สมัครมีทักษะและประสบการณ์ที่ตรงกับความต้องการเหล่านั้นหรือไม่? มีส่วนไหนที่ขาดไป?
3. สรุปจุดแข็งและจุดอ่อนของผู้สมัครเมื่อเทียบกับตำแหน่งงานนี้
4. จากการวิเคราะห์ทั้งหมด ให้คะแนนความเข้ากันได้จาก 1 (ไม่เหมาะเลย) ถึง 5 (เหมาะมาก)

จงตอบในรูปแบบ JSON เท่านั้น โดยมี key คือ "reasoning" และ "score":"""

    # สร้าง chat template สำหรับโมเดล
    chat = [
        {"role": "system", "content": "คุณคือผู้ช่วย HR อัจฉริยะที่วิเคราะห์และให้คะแนนความเข้ากันได้ระหว่างเรซูเม่กับตำแหน่งงาน คุณต้องตอบกลับเป็น JSON object ที่มี key 'reasoning' และ 'score' เท่านั้น"},
        {"role": "user", "content": prompt_template}
    ]
    return chat

# ทดลองสร้าง Prompt
sample_resume = "Experienced python developer with 5 years in data science, machine learning, and cloud platforms like AWS."
sample_job = "Seeking a senior data scientist with expertise in Python, scikit-learn, and AWS Sagemaker. 3+ years of experience required."

example_prompt = create_cot_prompt(sample_resume, sample_job)
print("--- ตัวอย่าง Prompt ที่จะส่งให้โมเดล ---")
print(json.dumps(example_prompt, indent=2, ensure_ascii=False))


--- ตัวอย่าง Prompt ที่จะส่งให้โมเดล ---
[
  {
    "role": "system",
    "content": "คุณคือผู้ช่วย HR อัจฉริยะที่วิเคราะห์และให้คะแนนความเข้ากันได้ระหว่างเรซูเม่กับตำแหน่งงาน คุณต้องตอบกลับเป็น JSON object ที่มี key 'reasoning' และ 'score' เท่านั้น"
  },
  {
    "role": "user",
    "content": "คุณคือ HR ที่เชี่ยวชาญ จงประเมินว่าเรซูเม่นี้เหมาะกับตำแหน่งงานนี้แค่ไหน\n\nตำแหน่งงาน: \"Seeking a senior data scientist with expertise in Python, scikit-learn, and AWS Sagemaker. 3+ years of experience required.\"\nเรซูเม่: \"Experienced python developer with 5 years in data science, machine learning, and cloud platforms like AWS.\"\n\nคิดวิเคราะห์เป็นขั้นตอนดังนี้:\n1. ทักษะหลักที่ตำแหน่งงานต้องการคืออะไร?\n2. ผู้สมัครมีทักษะและประสบการณ์ที่ตรงกับความต้องการเหล่านั้นหรือไม่? มีส่วนไหนที่ขาดไป?\n3. สรุปจุดแข็งและจุดอ่อนของผู้สมัครเมื่อเทียบกับตำแหน่งงานนี้\n4. จากการวิเคราะห์ทั้งหมด ให้คะแนนความเข้ากันได้จาก 1 (ไม่เหมาะเลย) ถึง 5 (เหมาะมาก)\n\nจงตอบในรูปแบบ JSON เท่านั้น โดยมี key คือ \"reasoni

In [10]:
# %% 5. การใช้ LLM สร้างฟีเจอร์

def generate_llm_features(df):
    """
    วน Loop ใน DataFrame เพื่อสร้างฟีเจอร์ reasoning และ score จาก LLM
    """
    results = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Generating LLM Features"):
        prompt = create_cot_prompt(row['resume_text'], row['job_description_text'])
        inputs = tokenizer.apply_chat_template(
            prompt, tokenize=True, add_generation_prompt=True, return_tensors="pt"
        ).to(DEVICE)

        # จำกัดความยาว input ป้องกัน error
        if inputs.shape[1] > 3072:
            inputs = inputs[:, -3072:]

        # สั่งให้โมเดลสร้างข้อความ
        with torch.no_grad():
            outputs = model_llm.generate(
                inputs,
                max_new_tokens=256,
                do_sample=False, # ใช้ do_sample=False เพื่อให้ผลลัพธ์คงที่
                pad_token_id=tokenizer.eos_token_id
            )

        response_text = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
        print(response_text)
        # Parse JSON ที่ได้จาก LLM
        try:
            start_index = response_text.find('{')
            end_index = response_text.rfind('}')
            json_part = response_text[start_index : end_index + 1]
            data = json.loads(json_part)
            results.append({
                'llm_reasoning': data.get('reasoning', ''),
                'llm_score': float(data.get('score', 0))
            })
        except (json.JSONDecodeError, AttributeError, ValueError):
            results.append({'llm_reasoning': '', 'llm_score': 0}) # ใส่ค่า default หากเกิดข้อผิดพลาด

    return pd.DataFrame(results)


# สร้างฟีเจอร์สำหรับข้อมูล Train
llm_features_train = generate_llm_features(train_df)
train_df_featured = pd.concat([train_df.reset_index(drop=True), llm_features_train], axis=1)

print("\nสร้างฟีเจอร์จาก LLM สำหรับ Train Set สำเร็จ")
print(train_df_featured[['llm_reasoning', 'llm_score', 'match_score']].head())


Generating LLM Features:   0%|          | 0/500 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


{
  "reasoning": "ผู้สมัครมีประสบการณ์ในการทำงานเกี่ยวกับการสังเกตและรายงานเหตุการณ์ที่มีความรุนแรงเกี่ยวกับการทำงานของคนงานเหมือนที่ระบุในเรซูเม่ แต่ไม่มีรายละเอียดเพิ่มเติมเกี่ยวกับความสามารถในการจัดการกับสถานการณ์ที่ยากลำบาก หรือการบริหารจัดการความเสี่ยงที่เกี่ยวข้องกับงาน ซึ่งเป็นทักษะที่สำคัญสำหรับตำแหน่งงานนี้ จุดแข็งของผู้สมัครคือการสังเกตและรายงานเหตุการณ์ แต่จุดอ่อนคือการจัดการกับสถานการณ์ที่ยากลำบาก ดังนั้น ผู้สมัครอาจจะสามารถทำหน้าที่ได้ แต่อาจต้องมีการฝึกฝนเพิ่มเติมในทักษะที่ขาดหายไป


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


{
  "reasoning": "ผู้สมัครมีประสบการณ์ในการทำงานเกี่ยวกับเทคโนโลยีจากเรซูเม่ที่ระบุไว้ แต่ไม่มีทักษะหรือประสบการณ์ที่เกี่ยวข้องกับการจัดการกฎหมายหรือการบริหารงานทางการเมือง ที่สำคัญสำหรับตำแหน่งงานที่ต้องการคือการเข้าใจและปฏิบัติตามคำสั่งศาลและการจัดการกับสถานการณ์ทางการเมือง ซึ่งไม่ได้ปรากฏในเรซูเม่ของผู้สมัคร จึงทำให้เราคาดว่าผู้สมัครอาจจะไม่สามารถดำเนินงานได้อย่างมีประสิทธิภาพในตำแหน่งงานที่ต้องการ",
  "score": 1
}


...

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


{
  "reasoning": "ตำแหน่งงานที่กำหนดคือ 'A woman fixes her hair.' ซึ่งหมายถึงการใช้บริการของผู้หญิงในการทำผม แต่เรซูเม่ที่ส่งมาคือ 'A man dries his hands.' ซึ่งไม่มีส่วนเกี่ยวข้องกับการดูแลผมหรือบริการที่เกี่ยวข้องกับผู้หญิง ดังนั้น ผู้สมัครไม่มีทักษะหรือประสบการณ์ที่ตรงกับความต้องการของตำแหน่งงาน จึงไม่สามารถทำงานในตำแหน่งนี้ได้ จุดแข็งของผู้สมัครไม่ได้เกี่ยวข้องกับตำแหน่งงานที่กำหนด จุดอ่อนคือขาดทักษะที่จำเป็นสำหรับตำแหน่งงาน ดังนั้น จึงให้คะแนนความเข้ากันได้ 1 จาก 5",
  "score": 1
}


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


{
  "reasoning": "ตำแหน่งงานที่ระบุมาเป็นภาพถ่ายของนักสกีที่ยืนบนมือในหิมะหน้ากล้องภาพยนตร์ ส่วนเรซูเม่ของผู้สมัครเป็นภาพของนักสกีที่แต่งกายด้วยกางเกงหิมะสีฟ้ากำลังกระโดดผ่านอากาศใกล้กับจุดกระโดด ทักษะหลักที่ตำแหน่งงานต้องการคือความสามารถในการทำงานกับกล้องและภาพถ่าย ซึ่งผู้สมัครมีทักษะนี้อยู่ แต่ขาดทักษะในการทำงานกับกล้องภาพยนตร์และมีการใช้เทคนิคการถ่ายภาพที่แตกต่างกัน จุดแข็งของผู้สมัครคือทักษะในการสกีและการกระโดดผ่านอากาศ แต่จุดอ่อนคือประสบการณ์ในการทำงานกับกล้องภาพยนตร์ ดังนั้น ผู้สมัครมีโอกาสที่จะเข้า


KeyboardInterrupt: 

In [ ]:
# %% 6. การสร้างฟีเจอร์เพิ่มเติม (Reasoning Embedding)

print("กำลังโหลดโมเดลสำหรับสร้าง Embedding...")
encoder_model = SentenceTransformer('all-MiniLM-L6-v2', device=DEVICE)

def create_reasoning_embeddings(df):
    """
    แปลงคอลัมน์ llm_reasoning เป็นเวกเตอร์ Embedding
    """
    # จัดการกับค่าที่หายไป (ถ้ามี)
    reasoning_texts = df['llm_reasoning'].fillna('').tolist()
    embeddings = encoder_model.encode(
        reasoning_texts,
        show_progress_bar=True,
        convert_to_numpy=True
    )
    # สร้าง DataFrame จาก embedding
    embedding_df = pd.DataFrame(embeddings, columns=[f'reason_embed_{i}' for i in range(embeddings.shape[1])])
    return embedding_df

# สร้าง Embedding สำหรับข้อมูล Train
reasoning_embeddings_train = create_reasoning_embeddings(train_df_featured)
final_train_df = pd.concat([train_df_featured, reasoning_embeddings_train], axis=1)


print("\nสร้าง Reasoning Embedding สำหรับ Train Set สำเร็จ")
print("ขนาดของ DataFrame สุดท้าย:", final_train_df.shape)
print(final_train_df.head())


In [ ]:
# %% 7. การฝึกโมเดล Regression สุดท้าย (LightGBM)

# กำหนดฟีเจอร์ (X) และเป้าหมาย (y)
features = ['llm_score'] + [col for col in final_train_df.columns if 'reason_embed_' in col]
target = 'match_score'

X_train = final_train_df[features]
y_train = final_train_df[target]

# กำหนดพารามิเตอร์สำหรับ LightGBM
lgb_params = {
    'objective': 'regression_l1', # MAE loss
    'metric': 'rmse',
    'n_estimators': 2000,
    'learning_rate': 0.01,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'lambda_l1': 0.1,
    'lambda_l2': 0.1,
    'num_leaves': 31,
    'verbose': -1,
    'n_jobs': -1,
    'seed': 42
}

print("กำลังฝึกโมเดล LightGBM สุดท้าย...")
final_model = lgb.LGBMRegressor(**lgb_params)
final_model.fit(X_train, y_train,
                eval_set=[(X_train, y_train)],
                eval_metric='rmse',
                callbacks=[lgb.early_stopping(100, verbose=False)])

print("ฝึกโมเดล LightGBM สำเร็จ!")


In [ ]:
# %% 8. การทำนายผลและสร้างไฟล์ Submission

print("--- เริ่มกระบวนการทำนายผลบน Test Set ---")

# 1. ใช้ LLM สร้างฟีเจอร์สำหรับ Test Set
llm_features_test = generate_llm_features(test_df)
test_df_featured = pd.concat([test_df.reset_index(drop=True), llm_features_test], axis=1)

# 2. สร้าง Reasoning Embedding สำหรับ Test Set
reasoning_embeddings_test = create_reasoning_embeddings(test_df_featured)
final_test_df = pd.concat([test_df_featured, reasoning_embeddings_test], axis=1)

# 3. เตรียมข้อมูลฟีเจอร์สำหรับทำนาย
X_test = final_test_df[features]

# 4. ทำนายผล
print("\nกำลังทำนาย match_score...")
predictions = final_model.predict(X_test)

# 5. สร้างไฟล์ Submission
submission_df = pd.DataFrame({
    'pair_id': final_test_df['pair_id'],
    'match_score': predictions
})

# Clip ค่าให้อยู่ในขอบเขต 1-5
submission_df['match_score'] = submission_df['match_score'].clip(0, 5)

print("\nตัวอย่างผลลัพธ์ในไฟล์ Submission:")
print(submission_df.head())

submission_df.to_csv("submission.csv", index=False)
print("\nสร้างไฟล์ submission.csv สำเร็จ!")

# ประเมินผล (ทำได้เพราะเรามี actual_score)
rmse = np.sqrt(mean_squared_error(final_test_df['actual_score'], predictions))
print(f"\nRoot Mean Squared Error (RMSE) บน Test Set: {rmse:.4f}")
